In [45]:
import sounddevice as sd
import numpy as np
import random
from scipy.io.wavfile import read

# Global variables
buffer_size = 1024  # 每个回调帧数
active_tracks = []  # 保存 (音频数据, 当前播放位置) 的列表

samplerate = None
current_position = 0  # 当前播放位置（全局同步）

# Function to load audio files
def load_audio(filename):
    global samplerate
    filename = "stem/" + filename  # 添加路径前缀
    print(f"Loading: {filename}")
    sr, data = read(filename)
    samplerate = sr if samplerate is None else samplerate
    data = data / np.max(np.abs(data), axis=0)  # 归一化到 [-1, 1]
    if len(data.shape) > 1:  # 转换为单声道
        data = np.mean(data, axis=1)
    return data

# Real-time callback for playback
def callback(outdata, frames, time, status):
    if status:
        print(status)
    global active_tracks, current_position

    mix = np.zeros(frames)  # 初始化混音缓冲区

    for i, (track, position) in enumerate(active_tracks):
        remaining_frames = len(track) - position  # 剩余帧数
        if remaining_frames >= frames:
            mix += track[position:position + frames]
            active_tracks[i] = (track, position + frames)  # 更新播放位置
        else:
            mix[:remaining_frames] += track[position:]
            # 如果开启循环播放，将音轨位置重置为0
            active_tracks[i] = (track, 0 + (frames - remaining_frames))  # 重新开始

    current_position += frames  # 更新全局位置
    outdata[:, 0] = mix  # 将混音写入输出缓冲区

    # 移除播放完成的音轨
    active_tracks[:] = [(track, pos) for track, pos in active_tracks if pos < len(track)]

# Function to add a track at the current position
def add_track(track):
    global current_position
    print("Adding a new track...")
    active_tracks.append((track, current_position))  # 设置开始位置为当前全局位置

# Function to remove the last track
def remove_track():
    global active_tracks
    if active_tracks:
        print("Removing the last added track...")
        track_list.append(active_tracks.pop())
    else:
        print("No tracks to remove!")

# Main function
def main():
    global samplerate, current_position,track_list
    # 加载音频文件
    track1 = load_audio("Tides of Ocean_m1_1.wav")
    track2 = load_audio("Tides of Ocean_m2_1.wav")
    track3 = load_audio("Tides of Ocean_chord_1.wav")
    track4 = load_audio("Tides of Ocean_bass_1.wav")
    track_list = [track1, track2, track3, track4]

    # 启动音频流
    print("Press '+' to add a track, '-' to remove the last track, and 'q' to quit.")
    stream = sd.OutputStream(callback=callback, samplerate=samplerate, channels=1, blocksize=buffer_size)

    try:
        with stream:
            while True:
                user_input = input("Enter command (+/-/q): ").strip()
                if user_input == "+":
                    if len(active_tracks) <= 3:
                        track_i = random.randrange(len(track_list))
                        add_track(track_list.pop(track_i))
                elif user_input == "-":
                    remove_track()
                elif user_input == "q":
                    print("Exiting program...")
                    break
                else:
                    print("Invalid input. Use '+' to add, '-' to remove, 'q' to quit.")
    except KeyboardInterrupt:
        print("\nPlayback stopped.")
    finally:
        print("Program exited.")

# Run the program
main()


Loading: stem/Tides of Ocean_m1_1.wav
Loading: stem/Tides of Ocean_m2_1.wav


C:\Users\Shangxuan\AppData\Local\Temp\ipykernel_8368\1520016488.py:18: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, data = read(filename)


Loading: stem/Tides of Ocean_chord_1.wav
Loading: stem/Tides of Ocean_bass_1.wav
Press '+' to add a track, '-' to remove the last track, and 'q' to quit.
Adding a new track...
Adding a new track...
Adding a new track...
Adding a new track...
Exiting program...
Program exited.


In [38]:
random.choice([0,3])

3